In [ ]:
import pandas as pd
import seaborn as sb
from sklearn.preprocessing import StandardScaler

## data pre-processing

In [ ]:
df = pd.read_csv('/Users/rumethsandinu/Study/Assessments/Year 2/DSGP/Datasets/final_cleaned_dataset/cleaned_annex.csv')
df.sample(5)

In [ ]:
df['Sale or Return'].value_counts()

In [ ]:
df = df.drop(['Date', 'Time', 'Item Code', 'CustomerId', 'Sale or Return', 'Loss Rate'], axis=1)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
for i in range(df.shape[0]):
    if df['Discount (Yes/No)'].values[i] == 'Yes':
        df['Discount (Yes/No)'].values[i] = 1
    else:
        df['Discount (Yes/No)'].values[i] = 0
        
df['Discount (Yes/No)'] = df['Discount (Yes/No)'].astype(int)
df.sample(5)  

In [ ]:
df.info()

In [ ]:
df['Discount (Yes/No)'].value_counts()

In [ ]:
df.iloc[:,[0, 1, 2]].corr()

In [ ]:
sb.pairplot(df.iloc[:,[0, 1, 2]])

In [ ]:
df.head()

In [ ]:
df['Discount (Yes/No)'] = df['Discount (Yes/No)'].astype(str)

In [ ]:
df['Quantity Sold (kilo)'].min(),df['Quantity Sold (kilo)'].max()

In [ ]:
df['Unit Selling Price (RMB/kg)'].min(), df['Unit Selling Price (RMB/kg)'].max()

In [ ]:
scaler = StandardScaler()
df['Quantity Sold (kilo)'] = scaler.fit_transform(df['Quantity Sold (kilo)'].values.reshape(-1, 1))
df['Unit Selling Price (RMB/kg)'] = scaler.fit_transform(df['Unit Selling Price (RMB/kg)'].values.reshape(-1, 1))
df.head()

In [ ]:
df = pd.get_dummies(df)
df.shape

In [ ]:
df.sample(5)

In [ ]:
from sklearn_extra.cluster import KMedoids
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
k = 100
kmedoids = KMedoids(n_clusters=k, random_state=0)
kmedoids.fit(df)

In [ ]:
labels = kmedoids.labels_
medoid_indices = kmedoids.medoid_indices_

# Randomly sample a subset of clusters to plot
sampled_clusters = np.random.choice(np.unique(labels), size=10, replace=False)

# Plot the sampled clusters
for i in sampled_clusters:
    cluster_points = df[labels == i]
    medoid = df[medoid_indices[i]]
    
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {i + 1}')
    plt.scatter(medoid[0], medoid[1], c='red', marker='x', s=200, label=f'Medoid {i + 1}')

plt.title('K-Medoids Clustering (Sampled Clusters)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()